In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
import csv
from sklearn import preprocessing
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.layers.core import Masking
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df = pd.merge(df3, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.sort_values(['user_id', 'time'], inplace=True)
df.reset_index(drop=True, inplace=True)
print('read success!')

read success!


In [20]:
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(5000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X = df[['user_id', 'time', 'click_times']]
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(4000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['product_category_0', 'product_category_1',
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((29, 1))))
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(70000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:30082788, :], columns=['advertiser_id_0', 'advertiser_id_1',
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, x_tmp], axis=1)
print('prepare success!')

(30082800, 1)
300828/300828 [==============================] - 4s 12us/step
(30082800, 1)
300828/300828 [==============================] - 4s 13us/step
(30082800, 1)
300828/300828 [==============================] - 3s 10us/step
prepare success!


In [59]:
%%time
def check_matrix(x):
    x_tmp = x.iloc[:,1:]
    x_tmp = x_tmp.groupby('time').sum()
    x_tmp = x_tmp.reset_index()
    tmp2 = np.zeros((91, 22))
    tmp2 = pd.DataFrame(tmp2)
    tmp2.columns = x_tmp.columns
    for i in range(x_tmp.shape[0]):
        tmp2.iloc[int(x_tmp.iloc[i, 0]) - 1, :] = x_tmp.iloc[i, :]
    return tmp2

X = X.groupby('user_id').apply(check_matrix)
X = X.reset_index(drop=True)
X = X.drop('time', axis = 1)
X = np.array(X).reshape((-1, 91, 21))

CPU times: user 2h 52min 5s, sys: 3min, total: 2h 55min 5s
Wall time: 2h 55min 56s


In [63]:
# X_ready = pd.DataFrame(X.reshape(-1, 21))

In [64]:
# X_ready.to_csv('data/train_preliminary/lstm2_5_straightsum.csv')

In [70]:
item_num = 10000
X_train, X_test = X[:item_num, :, :], X[800000:, :, :]
Y2_train, Y2_test = df1.iloc[:item_num, 2:3], df1.iloc[800000:, 2:3]
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(91, 21)))
model.add(LSTM(32))
model.add(Dense(20))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, Y2_train, epochs=20, use_multiprocessing=True, workers=4, shuffle=True)

Epoch 1/20
10000/10000 [==============================] - 22s 2ms/step - loss: 0.2696
Epoch 2/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2220
Epoch 3/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2217
Epoch 4/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2213
Epoch 5/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2198
Epoch 6/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2196
Epoch 7/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2198
Epoch 8/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2203
Epoch 9/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2195
Epoch 10/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2185
Epoch 11/20
10000/10000 [==============================] - 20s 2ms/step - loss: 0.2185
Epoch 12/20
10000/10000 [===========================

In [73]:
Y_predict = model.predict(X_test)
Y_predict = np.round(Y_predict)
ans = 0
for i in range(100000):
    if Y_predict[i] == Y2_test.iloc[i, 0]:
        ans += 1
print('accurency:', ans/100000)

accurency: 0.67229


In [92]:
item_num = 100000
X_train, X_test = X[:item_num, :, :], X[800000:, :, :]
Y2_train, Y2_test = df1.iloc[:item_num, 1:2], df1.iloc[800000:, 1:2]
Y2_train = pd.get_dummies(Y2_train['age'])
Y2_train = Y2_train.iloc[:,2]
Y2_train

0        0
1        0
2        0
3        0
4        0
        ..
99995    1
99996    0
99997    0
99998    0
99999    0
Name: 3, Length: 100000, dtype: uint8

In [93]:
model2 = Sequential()
model2.add(Masking(mask_value=0., input_shape=(91, 21)))
model2.add(LSTM(32))
model2.add(Dense(20))
model2.add(Dense(1))
model2.compile(loss='MSE', optimizer='adam')
model2.fit(X_train, Y2_train, epochs=10, use_multiprocessing=True, workers=4, shuffle=True)

Epoch 1/10
100000/100000 [==============================] - 213s 2ms/step - loss: 0.1745
Epoch 2/10
100000/100000 [==============================] - 215s 2ms/step - loss: 0.1737
Epoch 3/10
100000/100000 [==============================] - 215s 2ms/step - loss: 0.1734
Epoch 4/10
100000/100000 [==============================] - 213s 2ms/step - loss: 0.1732
Epoch 5/10
100000/100000 [==============================] - 215s 2ms/step - loss: 0.1731
Epoch 6/10
100000/100000 [==============================] - 213s 2ms/step - loss: 0.1729
Epoch 7/10
100000/100000 [==============================] - 213s 2ms/step - loss: 0.1728
Epoch 8/10
100000/100000 [==============================] - 216s 2ms/step - loss: 0.1728
Epoch 9/10
100000/100000 [==============================] - 213s 2ms/step - loss: 0.1727
Epoch 10/10
100000/100000 [==============================] - 213s 2ms/step - loss: 0.1726


In [94]:
Y_predict = model2.predict(X_train)
Y_predict = np.round(Y_predict)

In [95]:
ans = 0
for i in range(item_num):
    if Y_predict[i] == Y2_train.iloc[i]:
        ans += 1
print('accurency:', ans/item_num)

accurency: 0.77551
